In [2]:
import gym
from flowpipe import Graph, INode, Node, InputPlug, OutputPlug


In [ ]:
class Stream(INode):
    def __init__(self, **kwargs):
        super(Stream, self).__init__(**kwargs)
        self.data = []


    def add_data(self, new_data: List, key: Callable=None) -> None:
        if key is None:
            self.data.extend(new_data)
            return

        data_as_dict = {key(x):x for x in self.data}

        for record in new_data:
            # this may sometimes override existing records
            # but that's intentional as we only want one record per key
            data_as_dict[key(record)] = record

        self.data = list(data_as_dict.values())


    def get_data(self, drop=False):
        data_to_return = self.data[:]
        if drop:
            self.data = []
        return data_to_return
    
## Input streams

class StateAgent(Stream):
    def __init__(self, **kwargs):
        super(StateAgent, self).__init__(**kwargs)
        OutputPlug('agent_state', self)
    
    def compute(self) -> Dict:
        return {'agent_state': self.data}

class RewardAgent(Stream):
    def __init__(self, **kwargs):
        super(RewardAgent, self).__init__(**kwargs)
        OutputPlug('agent_reward', self)
    
    def compute(self) -> Dict:
        return {'agent_reward': self.data}

## Inner streams

class WeightPolicy(Stream):
    def __init__(self, **kwargs):
        super(WeightPolicy, self).__init__(**kwargs)
        InputPlug('weight_policy', self)
        OutputPlug('weight_policy', self)
    
    def compute(self, weight) -> Dict:
        self.add_data(weight)
        return {'weight_stream': self.data}

## Output streams

class ActionAgent(Stream):
    def __init__(self, **kwargs):
        super(ActionAgent, self).__init__(**kwargs)
        InputPlug('action_agent', self)
        OutputPlug('action_agent', self)

    def compute(self, action_agent: List) -> Dict:
        self.add_data(action_agent)
        return {'action_agent': self.data}    
    
## Processing Node


In [3]:
class ComputeAction(INode):
    """A node can be derived from the INode interface.

    The plugs are defined in the init method.
    The compute method received the inputs from any connected upstream nodes.
    """

    def __init__(self, state=None, reward=None, action_model=None, **kwargs):
        super(HireWorkers, self).__init__(**kwargs)
        InputPlug('state', self, state)
        InputPlug('reward', self, reward)
        InputPlug('action_model', self, action_model)
        OutputPlug('action', self)

    def compute(self, state,reward,action_model):
        return np.dot(action_model,state)


In [ ]:
graph = Graph(name='Agent')


In [5]:
a = {"1":2}
list(a.values())

[2]

In [6]:
{key(x):x for x in list(a.values())}

NameError: name 'key' is not defined